<a href="https://colab.research.google.com/github/bonillahermes/Data_Science_Projects/blob/main/ModelosClasPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hermes Yate Bonilla
**Data Scientist**
---

**Contact:**
- **Email:** [bonillahermes@gmail.com](mailto:bonillahermes@gmail.com)
- **LinkedIn:** [linkedin.com/in/bonillahermes](https://www.linkedin.com/in/bonillahermes/)
- **GitHub:** [github.com/bonillahermes](https://github.com/bonillahermes)
- **Webpage:** [bonillahermes.com](https://bonillahermes.com/)
---

# Modelos de Clasificación y Predicción

# Preliminares

In [ ]:
# Instalación de librerías
!pip install keras_tuner

In [ ]:
# Importación de librerías generales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.stats import f_oneway
from scipy.spatial.distance import mahalanobis

# Importación de bibliotecas de machine learning de scikit-learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, cross_val_score
from sklearn.metrics import (
    accuracy_score, confusion_matrix, precision_score, recall_score,
    roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score,
    mean_squared_error, r2_score, silhouette_score
)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

# Importación de XGBoost
import xgboost as xgb

# Importación de bibliotecas para modelos y redes neuronales
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Importación de Keras Tuner
import keras_tuner as kt

# Importación de bibliotecas para visualización interactiva
import plotly.graph_objs as go
import plotly.express as px

# Modelo 1

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Base.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
sheet_name = ('Mod1') # Nombres de las hoja
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar las primeras filas del DataFrame
df.head()

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

## Depuración

In [ ]:
# Calcular el porcentaje de valores faltantes para cada columna
missing_percentage = df.isnull().mean() * 100

# Filtrar las columnas que tienen menos del 30% de valores faltantes
columns_to_keep = missing_percentage[missing_percentage < 30].index

# Crear un nuevo DataFrame solo con las columnas seleccionadas
df = df[columns_to_keep]

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

## Imputación

In [ ]:
# Mantener 'NORDEMP' fuera de las columnas que serán imputadas
nuid_column = df['NORDEMP']
ciiu_column = df['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la imputación
df_for_imputation = df.drop(columns=['NORDEMP', 'CIIU 4'])

In [ ]:
# Imputar valores iniciales con KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df_for_imputation = pd.DataFrame(knn_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

# Imputar datos usando IterativeImputer (similar a MICE)
iterative_imputer = IterativeImputer(max_iter=10, random_state=42)
df_for_imputation = pd.DataFrame(iterative_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

In [ ]:
# Mostrar el número de valores faltantes por columna
df_for_imputation.isnull().sum()

In [ ]:
# Volver a añadir la columna 'NORDEMP' al DataFrame
df_for_imputation['NORDEMP'] = nuid_column
df_for_imputation['CIIU 4'] = ciiu_column

# Guardar cambios
df = df_for_imputation

In [ ]:
# Guardar el DataFrame resultante en un archivo Excel llamado Mod1.xlsx
output_file = 'Mod1.xlsx'
df.to_excel(output_file, index=False)

## Reducción de Dimensionalidad

### Preliminar

In [ ]:
# Función para calcular la distancia de Mahalanobis
def mahalanobis_distances(components):
    mean = np.mean(components, axis=0)
    cov = np.cov(components.T)
    inv_cov = np.linalg.inv(cov)
    distances = [mahalanobis(comp, mean, inv_cov) for comp in components]
    return np.array(distances)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod1_reduced = Mod1.drop(columns=['NORDEMP', 'CIIU 4'])

# Escalar las características
scaler = StandardScaler()
Mod1_scaled = scaler.fit_transform(Mod1_reduced)

# Aplicar PCA
pca = PCA(n_components= 10)
pca.fit(Mod1_scaled)

# Obtener la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_

# Crear un DataFrame para la varianza explicada
explained_variance_df = pd.DataFrame({
    'Component': range(1, len(explained_variance) + 1),
    'Explained Variance': explained_variance
})

# Mostrar la varianza explicada por cada componente
print(explained_variance_df)

### Cargar Base

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod1.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod1 = pd.read_excel(file_path)

In [ ]:
Mod1.head()

#### t-SNE

In [ ]:
# Mantener 'NORDEMP' y 'CIIU 4' fuera
nuid_column = Mod1['NORDEMP']
ciiu_column = Mod1['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la reducción
Mod1_red = Mod1.drop(columns=['NORDEMP', 'CIIU 4'])

# Preprocesamiento: eliminar filas con valores faltantes
Mod1_red.dropna(inplace=True)

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(Mod1_red)

# Crear un DataFrame con los resultados de t-SNE
df_tsne = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_tsne['NORDEMP'] = nuid_column
df_tsne['CIIU 4'] = ciiu_column

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', data=df_tsne, palette='viridis')
plt.title('t-SNE results')
plt.show()

#### DBSCAN

In [ ]:
# Aplicar DBSCAN para detectar outliers
dbscan = DBSCAN(eps=3, min_samples=10)  # Ajusta los parámetros según tus datos
df_tsne['outlier'] = dbscan.fit_predict(tsne_results)
df_tsne['index'] = Mod1_red.index

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='outlier', palette='viridis', data=df_tsne, legend='full')
plt.title('t-SNE results with outliers marked')
plt.show()

#### Eliminación Outliers

In [ ]:
# Mostrar los datos atípicos
outliers = df_tsne[df_tsne['outlier'] == -1]
print("Datos atípicos detectados:")
print(outliers)

In [ ]:
# Filtrar los índices de los datos que no son outliers
non_outliers_indices = df_tsne[df_tsne['outlier'] != -1]['index']

# Crear un nuevo DataFrame sin los outliers
Mod1_cleaned = Mod1.loc[non_outliers_indices]

# Guardar el DataFrame limpio en un nuevo archivo Excel
Mod1_cleaned.to_excel('Mod1_cleaned.xlsx', index=False)


In [ ]:
Mod1 = Mod1_cleaned

In [ ]:
Mod1.head()

In [ ]:
Mod1.isnull().sum()

## Modelos de Clasificación

### Descripción y recodificación

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod1_cleaned.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod1 = pd.read_excel(file_path)

In [ ]:
# Crear una tabla de frecuencias para la variable I2R5C1
tabla_frecuencias = Mod1['I2R5C1'].value_counts().sort_index()

# Crear un DataFrame con la tabla de frecuencias para mayor claridad
tabla_frecuencias_df = pd.DataFrame(tabla_frecuencias).reset_index()
tabla_frecuencias_df.columns = ['Productividad', 'Frecuencia']

# Mostrar la tabla de frecuencias
print(tabla_frecuencias_df)

In [ ]:
# Recodificar la variable I2R5C1: 1 se mantiene igual, 2 y 3 se recodifican a 0
Mod1['I2R5C1'] = Mod1['I2R5C1'].replace({2: 0, 3: 0})

In [ ]:
# Crear una tabla de frecuencias para la variable I2R5C1
tabla_frecuencias = Mod1['I2R5C1'].value_counts().sort_index()

# Crear un DataFrame con la tabla de frecuencias para mayor claridad
tabla_frecuencias_df = pd.DataFrame(tabla_frecuencias).reset_index()
tabla_frecuencias_df.columns = ['Productividad', 'Frecuencia']

# Mostrar la tabla de frecuencias
print(tabla_frecuencias_df)

### Modelos

In [ ]:
# Seleccionar las características (todas menos 'I2R5C1' y 'I2R5C1_recoded') y la variable objetivo ('I2R5C1_recoded')
X = Mod1.drop(columns=['I2R5C1'])
y = Mod1['I2R5C1']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Redes Neuronales

In [ ]:
# Definir la función de creación de modelo
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=564, step=32),
            activation=hp.Choice("activation", ["relu"])
        ))
        if hp.Boolean("dropout"):
            model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(
        loss=tf.keras.losses.binary_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    return model

In [ ]:
# Búsqueda de hiperparámetros
tuner = kt.RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=10,
    directory="/content/Tuning_Productividad",
    project_name="modelo_productividad"
)

In [ ]:
stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Entrenar el modelo con los mejores hiperparámetros
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

In [ ]:
# Evaluar el modelo
eval_result = model.evaluate(X_test_scaled, y_test)

# Precisión en los datos de prueba
accuracy = eval_result[1]
print('Precisión en los datos de prueba:', accuracy)

In [ ]:
# Guardar el modelo localmente
model.save('/content/modelo_productividad.h5')

#### Regresión Logística

In [ ]:
# Definir el modelo de Regresión Logística
logreg = LogisticRegression()

# Definir el grid de hiperparámetros
param_grid_logreg = {
    'C': [0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'liblinear', 'saga'],
}

# Ajustar el GridSearchCV para la regresión logística
grid_search_logreg = GridSearchCV(
    LogisticRegression(max_iter=10000),
    param_grid_logreg,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

In [ ]:
# Realizar la búsqueda de hiperparámetros
grid_search_logreg.fit(X_train_scaled, y_train)

In [ ]:
# Obtener los mejores hiperparámetros
best_logreg = grid_search_logreg.best_estimator_

In [ ]:
# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_logreg.fit(X_train_scaled, y_train)
y_pred_logreg = best_logreg.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Mejores hiperparámetros para Regresión Logística:")
print(grid_search_logreg.best_params_)
print("Reporte de clasificación para Regresión Logística:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
# Entrenar el modelo de Regresión Logística con los mejores hiperparámetros
best_logreg = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=5000)
best_logreg.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_logreg = best_logreg.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Reporte de clasificación para Regresión Logística con mejores hiperparámetros:")
print(classification_report(y_test, y_pred_logreg))

#### Random Forest

In [ ]:
# Definir el modelo de Random Forest
rf = RandomForestClassifier()

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Realizar la búsqueda de hiperparámetros
grid_search_rf = GridSearchCV(
    RandomForestClassifier(),
    param_grid_rf,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Realizar la búsqueda de hiperparámetros
grid_search_rf.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_rf = grid_search_rf.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_rf.fit(X_train_scaled, y_train)
y_pred_rf = best_rf.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Mejores hiperparámetros para Random Forest:")
print(grid_search_rf.best_params_)
print("Reporte de clasificación para Random Forest:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
# Entrenar el modelo de Random Forest con los mejores hiperparámetros
best_rf = RandomForestClassifier(
    criterion='gini',
    max_depth=30,
    max_features='sqrt',
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=100
)
best_rf.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_rf = best_rf.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Reporte de clasificación para Random Forest con mejores hiperparámetros:")
print(classification_report(y_test, y_pred_rf))

#### XGBOOST

In [ ]:
# Definir el modelo de XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
# Definir el grid de hiperparámetros
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Realizar la búsqueda de hiperparámetros
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_xgb = grid_search_xgb.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_xgb.fit(X_train_scaled, y_train)
y_pred_xgb = best_xgb.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Mejores hiperparámetros para XGBoost:")
print(grid_search_xgb.best_params_)
print("Reporte de clasificación para XGBoost:")
print(classification_report(y_test, y_pred_xgb))


In [ ]:
# Entrenar el modelo de XGBoost con los mejores hiperparámetros
best_xgb = xgb.XGBClassifier(
    colsample_bytree=0.7,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100,
    subsample=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
best_xgb.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_xgb = best_xgb.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Reporte de clasificación para XGBoost con mejores hiperparámetros:")
print(classification_report(y_test, y_pred_xgb))

# Modelo 2

## Preliminares

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Base.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
sheet_name = ('Mod2') # Nombres de las hoja
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar las primeras filas del DataFrame
df.head()

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

### Depuración

In [ ]:
# Calcular el porcentaje de valores faltantes para cada columna
missing_percentage = df.isnull().mean() * 100

# Filtrar las columnas que tienen menos del 30% de valores faltantes
columns_to_keep = missing_percentage[missing_percentage < 30].index

# Crear un nuevo DataFrame solo con las columnas seleccionadas
df = df[columns_to_keep]

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

### Imputación

In [ ]:
# Mantener 'NORDEMP' fuera de las columnas que serán imputadas
nuid_column = df['NORDEMP']
ciiu_column = df['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la imputación
df_for_imputation = df.drop(columns=['NORDEMP', 'CIIU 4'])

In [ ]:
# Imputar valores iniciales con KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df_for_imputation = pd.DataFrame(knn_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

# Imputar datos usando IterativeImputer (similar a MICE)
iterative_imputer = IterativeImputer(max_iter=10, random_state=42)
df_for_imputation = pd.DataFrame(iterative_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

In [ ]:
# Mostrar el número de valores faltantes por columna
df_for_imputation.isnull().sum()

In [ ]:
# Volver a añadir la columna 'NORDEMP' al DataFrame
df_for_imputation['NORDEMP'] = nuid_column
df_for_imputation['CIIU 4'] = ciiu_column

# Guardar cambios
df = df_for_imputation

In [ ]:
# Guardar el DataFrame resultante en un archivo Excel llamado Mod2.xlsx
output_file = 'Mod2.xlsx'
df.to_excel(output_file, index=False)

## Reducción de Dimensionalidad

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod2.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod2 = pd.read_excel(file_path)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod2_reduced = Mod2.drop(columns=['NORDEMP', 'CIIU 4'])

# Escalar las características
scaler = StandardScaler()
Mod2_scaled = scaler.fit_transform(Mod2_reduced)

# Aplicar PCA
pca = PCA(n_components= 10)
pca.fit(Mod2_scaled)

# Obtener la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_

# Crear un DataFrame para la varianza explicada
explained_variance_df = pd.DataFrame({
    'Component': range(1, len(explained_variance) + 1),
    'Explained Variance': explained_variance
})

# Mostrar la varianza explicada por cada componente
print(explained_variance_df)

In [ ]:
# Graficar la varianza explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance.cumsum(), marker='o', linestyle='--')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada Acumulada por el PCA')
plt.grid()
plt.show()

#### t-SNE

In [ ]:
# Mantener 'NORDEMP' y 'CIIU 4' fuera
nuid_column = Mod2['NORDEMP']
ciiu_column = Mod2['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la reducción
Mod2_red = Mod2.drop(columns=['NORDEMP', 'CIIU 4'])

# Preprocesamiento: eliminar filas con valores faltantes
Mod2_red.dropna(inplace=True)

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(Mod2_red)

# Crear un DataFrame con los resultados de t-SNE
df_tsne = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_tsne['NORDEMP'] = nuid_column
df_tsne['CIIU 4'] = ciiu_column

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', data=df_tsne, palette='viridis')
plt.title('t-SNE results')
plt.show()

#### DBSCAN

In [ ]:
# Aplicar DBSCAN para detectar outliers
dbscan = DBSCAN(eps=3, min_samples=10)  # Ajusta los parámetros según tus datos
df_tsne['outlier'] = dbscan.fit_predict(tsne_results)
df_tsne['index'] = Mod2_red.index

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='outlier', palette='viridis', data=df_tsne, legend='full')
plt.title('t-SNE results with outliers marked')
plt.show()

#### Eliminación de Outliers

In [ ]:
# Mostrar los datos atípicos
outliers = df_tsne[df_tsne['outlier'] == -1]
print("Datos atípicos detectados:")
print(outliers)

In [ ]:
# Filtrar los índices de los datos que no son outliers
non_outliers_indices = df_tsne[df_tsne['outlier'] != -1]['index']

# Crear un nuevo DataFrame sin los outliers
Mod2_cleaned = Mod2.loc[non_outliers_indices]

# Guardar el DataFrame limpio en un nuevo archivo Excel
Mod2_cleaned.to_excel('Mod2_cleaned.xlsx', index=False)


In [ ]:
Mod2 = Mod2_cleaned

## Modelos de Clasificación

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod2_cleaned.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod2 = pd.read_excel(file_path)

In [ ]:
# Crear una tabla de frecuencias para la variable I10R7C1
tabla_frecuencias = Mod2['I10R7C1'].value_counts().sort_index()

# Crear un DataFrame con la tabla de frecuencias para mayor claridad
tabla_frecuencias_df = pd.DataFrame(tabla_frecuencias).reset_index()
tabla_frecuencias_df.columns = ['Incertidumbre', 'Frecuencia']

# Mostrar la tabla de frecuencias
print(tabla_frecuencias_df)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod2_reduced = Mod2.drop(columns=['NORDEMP', 'CIIU 4'])

# Seleccionar las características (todas menos 'I10R7C1') y la variable objetivo ('I10R7C1')
X = Mod2_reduced.drop(columns=['I10R7C1'])
y = Mod2_reduced['I10R7C1']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Definir el modelo de Regresión Logística Multinomial
logreg = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=5000)

# Definir el grid de hiperparámetros
param_grid_logreg = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'l1_ratio': [0, 0.5, 1]  # Solo aplicable si la penalización es 'elasticnet'
}

# Realizar la búsqueda de hiperparámetros
grid_search_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_logreg.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_logreg = grid_search_logreg.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_logreg.fit(X_train_scaled, y_train)
y_pred_logreg = best_logreg.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Mejores hiperparámetros para Regresión Logística Multinomial:")
print(grid_search_logreg.best_params_)
print("Reporte de clasificación para Regresión Logística Multinomial:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
# Entrenar el modelo de Regresión Logística Multinomial con los mejores hiperparámetros
best_logreg = LogisticRegression(
    C=0.01,
    penalty='l1',
    solver='saga',
    multi_class='multinomial',
    max_iter=5000
)
best_logreg.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_logreg = best_logreg.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Reporte de clasificación para Regresión Logística Multinomial con mejores hiperparámetros:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
# Definir el modelo de Random Forest
rf = RandomForestClassifier()

# Definir el grid de hiperparámetros
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Realizar la búsqueda de hiperparámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_rf = grid_search_rf.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_rf.fit(X_train_scaled, y_train)
y_pred_rf = best_rf.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Mejores hiperparámetros para Random Forest:")
print(grid_search_rf.best_params_)
print("Reporte de clasificación para Random Forest:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
# Entrenar el modelo de Random Forest con los mejores hiperparámetros
best_rf = RandomForestClassifier(
    criterion='gini',
    max_depth=20,
    max_features='auto',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=300
)
best_rf.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_rf = best_rf.predict(X_test_scaled)

# Mostrar el reporte de clasificación
print("Reporte de clasificación para Random Forest con mejores hiperparámetros:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Seleccionar las características (todas menos 'I10R7C1') y la variable objetivo ('I10R7C1')
X = Mod2_reduced.drop(columns=['I10R7C1'])
y = Mod2_reduced['I10R7C1']

# Recodificar las etiquetas de '1, 2, 3' a '0, 1, 2'
y = y - 1

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir la función de creación de modelo
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))
        if hp.Boolean("dropout"):
            model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(3, activation="softmax"))  # 3 clases para clasificación multiclase
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        metrics=['accuracy']
    )
    return model

# Búsqueda de hiperparámetros
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    directory="/content/tuning_neural_network",
    project_name="NN_tuning"
)

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Entrenar el modelo con los mejores hiperparámetros
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Evaluar el modelo
eval_result = model.evaluate(X_test_scaled, y_test)

# Precisión en los datos de prueba
accuracy = eval_result[1]
print('Precisión en los datos de prueba:', accuracy)

# Realizar predicciones
y_pred_nn = model.predict(X_test_scaled)
y_pred_nn_classes = y_pred_nn.argmax(axis=-1)

# Mostrar el reporte de clasificación
print("Reporte de clasificación para Redes Neuronales con mejores hiperparámetros:")
print(classification_report(y_test, y_pred_nn_classes))


# Modelo 3

## Preliminares

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Base.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
sheet_name = ('Mod3') # Nombres de las hoja
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar las primeras filas del DataFrame
df.head()

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

### Depuración

In [ ]:
# Calcular el porcentaje de valores faltantes para cada columna
missing_percentage = df.isnull().mean() * 100

# Filtrar las columnas que tienen menos del 30% de valores faltantes
columns_to_keep = missing_percentage[missing_percentage < 30].index

# Crear un nuevo DataFrame solo con las columnas seleccionadas
df = df[columns_to_keep]

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

### Imputación

In [ ]:
# Mantener 'NORDEMP' fuera de las columnas que serán imputadas
nuid_column = df['NORDEMP']
ciiu_column = df['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la imputación
df_for_imputation = df.drop(columns=['NORDEMP', 'CIIU 4'])

In [ ]:
# Imputar valores iniciales con KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df_for_imputation = pd.DataFrame(knn_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

# Imputar datos usando IterativeImputer (similar a MICE)
iterative_imputer = IterativeImputer(max_iter=10, random_state=42)
df_for_imputation = pd.DataFrame(iterative_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

In [ ]:
# Mostrar el número de valores faltantes por columna
df_for_imputation.isnull().sum()

In [ ]:
# Volver a añadir la columna 'NORDEMP' al DataFrame
df_for_imputation['NORDEMP'] = nuid_column
df_for_imputation['CIIU 4'] = ciiu_column

# Guardar cambios
df = df_for_imputation

In [ ]:
# Guardar el DataFrame resultante en un archivo Excel llamado Mod3.xlsx
output_file = 'Mod3.xlsx'
df.to_excel(output_file, index=False)

## Reducción de dimensionalidad

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod3.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod3 = pd.read_excel(file_path)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod3_reduced = Mod3.drop(columns=['NORDEMP', 'CIIU 4'])

# Escalar las características
scaler = StandardScaler()
Mod3_scaled = scaler.fit_transform(Mod3_reduced)

# Aplicar PCA
pca = PCA(n_components= 10)
pca.fit(Mod3_scaled)

# Obtener la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_

# Crear un DataFrame para la varianza explicada
explained_variance_df = pd.DataFrame({
    'Component': range(1, len(explained_variance) + 1),
    'Explained Variance': explained_variance
})

# Mostrar la varianza explicada por cada componente
print(explained_variance_df)

In [ ]:
# Graficar la varianza explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance.cumsum(), marker='o', linestyle='--')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada Acumulada por el PCA')
plt.grid()
plt.show()

### t-SNE

In [ ]:
# Mantener 'NORDEMP' y 'CIIU 4' fuera
nuid_column = Mod3['NORDEMP']
ciiu_column = Mod3['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la reducción
Mod3_red = Mod3.drop(columns=['NORDEMP', 'CIIU 4'])

# Preprocesamiento: eliminar filas con valores faltantes
Mod3_red.dropna(inplace=True)

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(Mod3_red)

# Crear un DataFrame con los resultados de t-SNE
df_tsne = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_tsne['NORDEMP'] = nuid_column
df_tsne['CIIU 4'] = ciiu_column

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', data=df_tsne, palette='viridis')
plt.title('t-SNE results')
plt.show()

### DBSCAN

In [ ]:
# Aplicar DBSCAN para detectar outliers
dbscan = DBSCAN(eps=3, min_samples=10)  # Ajusta los parámetros según tus datos
df_tsne['outlier'] = dbscan.fit_predict(tsne_results)
df_tsne['index'] = Mod3_red.index

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='outlier', palette='viridis', data=df_tsne, legend='full')
plt.title('t-SNE results with outliers marked')
plt.show()

### Eliminación de outliers

In [ ]:
# Mostrar los datos atípicos
outliers = df_tsne[df_tsne['outlier'] == -1]
print("Datos atípicos detectados:")
print(outliers)

In [ ]:
# Filtrar los índices de los datos que no son outliers
non_outliers_indices = df_tsne[df_tsne['outlier'] != -1]['index']

# Crear un nuevo DataFrame sin los outliers
Mod3_cleaned = Mod3.loc[non_outliers_indices]

# Guardar el DataFrame limpio en un nuevo archivo Excel
Mod3_cleaned.to_excel('Mod3_cleaned.xlsx', index=False)

## Modelos de Predicción

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod3_cleaned.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod3 = pd.read_excel(file_path)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod3 = Mod3.drop(columns=['NORDEMP', 'CIIU 4'])

# Seleccionar las características (todas menos 'II1R3C2') y la variable objetivo ('II1R3C2')
X = Mod3.drop(columns=['II1R3C2'])
y = Mod3['II1R3C2']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Entrenar el modelo de Regresión Lineal
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_lin_reg = lin_reg.predict(X_test_scaled)

# Evaluar el modelo
mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
r2_lin_reg = r2_score(y_test, y_pred_lin_reg)

print("Regresión Lineal")
print("Mean Squared Error:", mse_lin_reg)
print("R2 Score:", r2_lin_reg)

In [ ]:
# Definir el modelo de Random Forest
rf = RandomForestRegressor()

# Definir el grid de hiperparámetros
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizar la búsqueda de hiperparámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_rf = grid_search_rf.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_rf.fit(X_train_scaled, y_train)
y_pred_rf = best_rf.predict(X_test_scaled)

# Evaluar el modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest")
print("Mejores hiperparámetros:", grid_search_rf.best_params_)
print("Mean Squared Error:", mse_rf)
print("R2 Score:", r2_rf)

In [ ]:
# Crear el modelo de Random Forest con los mejores hiperparámetros
best_rf = RandomForestRegressor(
    max_depth=20,
    max_features='auto',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

# Entrenar el modelo
best_rf.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_rf = best_rf.predict(X_test_scaled)

# Evaluar el modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest")
print("Mean Squared Error:", mse_rf)
print("R2 Score:", r2_rf)

In [ ]:
# Definir el modelo de XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Definir el grid de hiperparámetros
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Realizar la búsqueda de hiperparámetros
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_xgb.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_xgb = grid_search_xgb.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_xgb.fit(X_train_scaled, y_train)
y_pred_xgb = best_xgb.predict(X_test_scaled)

# Evaluar el modelo
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost")
print("Mejores hiperparámetros:", grid_search_xgb.best_params_)
print("Mean Squared Error:", mse_xgb)
print("R2 Score:", r2_xgb)


In [ ]:
# Crear el modelo de XGBoost con los mejores hiperparámetros
best_xgb = xgb.XGBRegressor(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=5,
    n_estimators=300,
    subsample=0.7,
    objective='reg:squarederror',
    random_state=42
)

# Entrenar el modelo
best_xgb.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_xgb = best_xgb.predict(X_test_scaled)

# Evaluar el modelo
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost")
print("Mean Squared Error:", mse_xgb)
print("R2 Score:", r2_xgb)

In [ ]:
# Definir la función de creación de modelo
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))
        if hp.Boolean("dropout"):
            model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(1))  # Salida para regresión
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        metrics=['mean_squared_error']
    )
    return model

# Búsqueda de hiperparámetros
tuner = kt.RandomSearch(
    build_model,
    objective="val_mean_squared_error",
    max_trials=20,
    directory="/content/tuning_neural_network",
    project_name="NN_tuning"
)

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Entrenar el modelo con los mejores hiperparámetros
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Evaluar el modelo
eval_result = model.evaluate(X_test_scaled, y_test)

# Precisión en los datos de prueba
mse = eval_result[0]
r2 = r2_score(y_test, model.predict(X_test_scaled))

print('Mean Squared Error en los datos de prueba:', mse)
print('R2 Score en los datos de prueba:', r2)

# Modelo 4

## Preliminares

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Base.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
sheet_name = ('Mod4') # Nombres de las hoja
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar las primeras filas del DataFrame
df.head()

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

### Depuración

In [ ]:
# Calcular el porcentaje de valores faltantes para cada columna
missing_percentage = df.isnull().mean() * 100

# Filtrar las columnas que tienen menos del 30% de valores faltantes
columns_to_keep = missing_percentage[missing_percentage < 30].index

# Crear un nuevo DataFrame solo con las columnas seleccionadas
df = df[columns_to_keep]

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

### Imputación

In [ ]:
# Mantener 'NORDEMP' fuera de las columnas que serán imputadas
nuid_column = df['NORDEMP']
ciiu_column = df['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la imputación
df_for_imputation = df.drop(columns=['NORDEMP', 'CIIU 4'])

In [ ]:
# Imputar valores iniciales con KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df_for_imputation = pd.DataFrame(knn_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

# Imputar datos usando IterativeImputer (similar a MICE)
iterative_imputer = IterativeImputer(max_iter=10, random_state=42)
df_for_imputation = pd.DataFrame(iterative_imputer.fit_transform(df_for_imputation), columns=df_for_imputation.columns)

In [ ]:
# Mostrar el número de valores faltantes por columna
df_for_imputation.isnull().sum()

In [ ]:
# Volver a añadir la columna 'NORDEMP' al DataFrame
df_for_imputation['NORDEMP'] = nuid_column
df_for_imputation['CIIU 4'] = ciiu_column

# Guardar cambios
df = df_for_imputation

In [ ]:
# Guardar el DataFrame resultante en un archivo Excel llamado Mod4.xlsx
output_file = 'Mod4.xlsx'
df.to_excel(output_file, index=False)

## Reducción de dimensionalidad

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod4.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod4 = pd.read_excel(file_path)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod4_reduced = Mod4.drop(columns=['NORDEMP', 'CIIU 4'])

# Escalar las características
scaler = StandardScaler()
Mod4_scaled = scaler.fit_transform(Mod4_reduced)

# Aplicar PCA
pca = PCA(n_components= 10)
pca.fit(Mod4_scaled)

# Obtener la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_

# Crear un DataFrame para la varianza explicada
explained_variance_df = pd.DataFrame({
    'Component': range(1, len(explained_variance) + 1),
    'Explained Variance': explained_variance
})

# Mostrar la varianza explicada por cada componente
print(explained_variance_df)

### t-SNE

In [ ]:
# Mantener 'NORDEMP' y 'CIIU 4' fuera
nuid_column = Mod4['NORDEMP']
ciiu_column = Mod4['CIIU 4']

# Eliminar 'NORDEMP' y 'CIIU 4' del DataFrame para la reducción
Mod4_red = Mod4.drop(columns=['NORDEMP', 'CIIU 4'])

# Preprocesamiento: eliminar filas con valores faltantes
Mod4_red.dropna(inplace=True)

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(Mod4_red)

# Crear un DataFrame con los resultados de t-SNE
df_tsne = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_tsne['NORDEMP'] = nuid_column
df_tsne['CIIU 4'] = ciiu_column

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', data=df_tsne, palette='viridis')
plt.title('t-SNE results')
plt.show()

### DBSCAN

In [ ]:
# Aplicar DBSCAN para detectar outliers
dbscan = DBSCAN(eps=3, min_samples=10)  # Ajusta los parámetros según tus datos
df_tsne['outlier'] = dbscan.fit_predict(tsne_results)
df_tsne['index'] = Mod4_red.index

# Visualizar los resultados
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='outlier', palette='viridis', data=df_tsne, legend='full')
plt.title('t-SNE results with outliers marked')
plt.show()

### Eliminación de outliers

In [ ]:
# Mostrar los datos atípicos
outliers = df_tsne[df_tsne['outlier'] == -1]
print("Datos atípicos detectados:")
print(outliers)

In [ ]:
# Filtrar los índices de los datos que no son outliers
non_outliers_indices = df_tsne[df_tsne['outlier'] != -1]['index']

# Crear un nuevo DataFrame sin los outliers
Mod4_cleaned = Mod4.loc[non_outliers_indices]

# Guardar el DataFrame limpio en un nuevo archivo Excel
Mod4_cleaned.to_excel('Mod4_cleaned.xlsx', index=False)

## Modelos de Predicción

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Mod4_cleaned.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
Mod4 = pd.read_excel(file_path)

In [ ]:
# Eliminar las columnas 'NORDEMP' y 'CIIU 4'
Mod4 = Mod4.drop(columns=['NORDEMP', 'CIIU 4'])

# Seleccionar las características (todas menos 'I3R2C1') y la variable objetivo ('I3R2C1')
X = Mod4.drop(columns=['I3R2C1'])
y = Mod4['I3R2C1']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Entrenar el modelo de Regresión Lineal
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_lin_reg = lin_reg.predict(X_test_scaled)

# Evaluar el modelo
mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
r2_lin_reg = r2_score(y_test, y_pred_lin_reg)

print("Regresión Lineal")
print("Mean Squared Error:", mse_lin_reg)
print("R2 Score:", r2_lin_reg)

In [ ]:
# Definir el modelo de Random Forest
rf = RandomForestRegressor()

# Definir el grid de hiperparámetros
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizar la búsqueda de hiperparámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_rf = grid_search_rf.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_rf.fit(X_train_scaled, y_train)
y_pred_rf = best_rf.predict(X_test_scaled)

# Evaluar el modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest")
print("Mejores hiperparámetros:", grid_search_rf.best_params_)
print("Mean Squared Error:", mse_rf)
print("R2 Score:", r2_rf)

In [ ]:
# Crear el modelo de Random Forest con los mejores hiperparámetros
best_rf = RandomForestRegressor(
    max_depth=30,
    max_features='auto',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

# Entrenar el modelo
best_rf.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_rf = best_rf.predict(X_test_scaled)

# Evaluar el modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest")
print("Mean Squared Error:", mse_rf)
print("R2 Score:", r2_rf)

In [ ]:
# Definir el modelo de XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Definir el grid de hiperparámetros
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Realizar la búsqueda de hiperparámetros
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_xgb.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_xgb = grid_search_xgb.best_estimator_

# Entrenar y evaluar el modelo con los mejores hiperparámetros
best_xgb.fit(X_train_scaled, y_train)
y_pred_xgb = best_xgb.predict(X_test_scaled)

# Evaluar el modelo
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost")
print("Mejores hiperparámetros:", grid_search_xgb.best_params_)
print("Mean Squared Error:", mse_xgb)
print("R2 Score:", r2_xgb)


In [ ]:
# Crear el modelo de XGBoost con los mejores hiperparámetros
best_xgb = xgb.XGBRegressor(
    colsample_bytree=0.8,
    learning_rate=0.2,
    max_depth=7,
    n_estimators=200,
    subsample=0.7,
    objective='reg:squarederror',
    random_state=42
)

# Entrenar el modelo
best_xgb.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_xgb = best_xgb.predict(X_test_scaled)

# Evaluar el modelo
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost")
print("Mean Squared Error:", mse_xgb)
print("R2 Score:", r2_xgb)

In [ ]:
# Definir la función de creación de modelo
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))
        if hp.Boolean("dropout"):
            model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(1))  # Salida para regresión
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        metrics=['mean_squared_error']
    )
    return model

# Búsqueda de hiperparámetros
tuner = kt.RandomSearch(
    build_model,
    objective="val_mean_squared_error",
    max_trials=20,
    directory="/content/tuning_neural_network2",
    project_name="NN_tuning"
)

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Entrenar el modelo con los mejores hiperparámetros
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Evaluar el modelo
eval_result = model.evaluate(X_test_scaled, y_test)

# Precisión en los datos de prueba
mse = eval_result[0]
r2 = r2_score(y_test, model.predict(X_test_scaled))

print('Mean Squared Error en los datos de prueba:', mse)
print('R2 Score en los datos de prueba:', r2)